# ESPN March Madness Scrapers

## Initialization

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

## Scraper 1

Scraper collects all the 364 teams involved in the NAACP as well as links that will be modified and scraped in the second scraper. Scrapes one page and one table.

In [2]:
# Set up the WebDriver
driver = webdriver.Chrome()

# URL for ESPN 
url = 'https://www.espn.com/mens-college-basketball/bpi'


# Open Fox News and searches for query
driver.get(url)

# Allow the page to load
time.sleep(3)

# Optionally, click to load more content (e.g., click the button 12 times)
for _ in range(100):
    try:
        load_button = driver.find_element(By.LINK_TEXT, "Show More")
        load_button.click()
        time.sleep(2)  # Wait for content to load
    except:
        # In case the "load-more" button is not found or clickable, pass
        pass

# Find the articles
table = driver.find_elements(By.CLASS_NAME, "Table__TBODY")

# Initialize lists to store headlines and URLs
team_name = []
urls = []

# Loop through each article and extract the headline and URL
for row in table:
    try:
        # Find individual cells or elements within the row, depending on the structure of the table
        cells = row.find_elements(By.TAG_NAME, "tr")  # Get each row in the table
        
        for cell in cells:
            # Extract the text from the cell (or row) and URL (if available)
            name = cell.text
            if name != 'just in':
                team_name.append(name)
                
                # Attempt to extract the URL if available
                link = cell.find_element(By.TAG_NAME, "a").get_attribute("href")
                urls.append(link)
    except Exception as e:
        print(f"Error processing row: {e}")

# Create a DataFrame with the headlines and URLs
df = pd.DataFrame({
    'Team_name': team_name[:-1],
    'URL': urls
})

# Print the resulting DataFrame
print(df)

# Close the browser
driver.quit()


Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)
Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)
Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)


Error processing row: Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: chrome=134.0.6998.44); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100ccbb78 cxxbridge1$str$ptr + 2778912
1   chromedriver                        0x0000000100cc41b0 cxxbridge1$str$ptr + 2747736
2   chromedriver                        0x0000000100819e24 cxxbridge1$string$len + 92932
3   chromedriver                        0x0000000100861158 cxxbridge1$string$len + 384568
4   chromedriver                        0x0000000100856800 cxxbridge1$string$len + 341216
5   chromedriver                        0x00000001008a2500 cxxbridge1$string$len + 651744
6   chromedriver                        0x00000001008552e4 cxxbridge1$string$len + 335812
7   chromedriver                        0x0000000100c91d04 cxxbridg

In [3]:
len(urls)

364

In [4]:
len(team_name)

365

In [5]:
'''df = pd.DataFrame({
    'Team_name': team_name[:-1],
    'URL': urls
})'''


"df = pd.DataFrame({\n    'Team_name': team_name[:-1],\n    'URL': urls\n})"

I clean up the dataframe.

In [6]:
df['Conference'] = df['Team_name'].str.partition(sep='\n')[2]
df['Team_name'] = df['Team_name'].str.partition(sep='\n')[0]

In [7]:
df = df.sort_values(by='Team_name', ascending=True).reset_index(drop=True)
df.head(20)

,Team_name,URL,Conference
0,Abilene Christian Wildcats,https://www.espn.com/mens-college-basketball/t...,WAC
1,Air Force Falcons,https://www.espn.com/mens-college-basketball/t...,Mountain West
2,Akron Zips,https://www.espn.com/mens-college-basketball/t...,MAC
3,Alabama A&M Bulldogs,https://www.espn.com/mens-college-basketball/t...,SWAC
4,Alabama Crimson Tide,https://www.espn.com/mens-college-basketball/t...,SEC
5,Alabama State Hornets,https://www.espn.com/mens-college-basketball/t...,SWAC
6,Alcorn State Braves,https://www.espn.com/mens-college-basketball/t...,SWAC
7,American University Eagles,https://www.espn.com/mens-college-basketball/t...,Patriot
8,App State Mountaineers,https://www.espn.com/mens-college-basketball/t...,Sun Belt
9,Arizona State Sun Devils,https://www.espn.com/mens-college-basketball/t...,Big 12


https://www.espn.com/mens-college-basketball/team/_/id/248/houston-cougars

https://www.espn.com/mens-college-basketball/team/schedule/_/id/248

In [10]:
# df['Schedule_link'] = df['URL'].str.replace('/_/', '/schedule' + '/_/').str.replace(r'/id/\d+/', '/schedule' + '')
df['Schedule_link'] =  df['URL'].apply(lambda url: re.sub(r'(https://www\.espn\.com/mens-college-basketball/team)/_(/id/\d+)', r'\1/schedule/_\2/season/2024', url))

I modify the links to creaet a series of links for each team's schedule. Additional parameters were added above for the season.

In [12]:
%matplotlib inline
import random

print(df['Schedule_link'][random.randint(0, len(df))])

https://www.espn.com/mens-college-basketball/team/schedule/_/id/301/season/2024/san-diego-toreros


The above cell tests that the data transformation was successful.

We must use Selenium because ESPN blocks the requests library.

In [14]:
pd.set_option('display.max_colwidth', 180)
df['Schedule_link'].head()

0    https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/season/2024/abilene-christian-wildcats
1             https://www.espn.com/mens-college-basketball/team/schedule/_/id/2005/season/2024/air-force-falcons
2                    https://www.espn.com/mens-college-basketball/team/schedule/_/id/2006/season/2024/akron-zips
3           https://www.espn.com/mens-college-basketball/team/schedule/_/id/2010/season/2024/alabama-am-bulldogs
4           https://www.espn.com/mens-college-basketball/team/schedule/_/id/333/season/2024/alabama-crimson-tide
Name: Schedule_link, dtype: object

## Scraper 2

For reasons unknown, a large number of teams escaped our scraper on first screening. We will scrape them again and concatenate our dataframe until all 364 teams are collected as well as the information pertinent to their schedules.

In [15]:
%%time

driver = webdriver.Chrome()
data = []  # Initialize an empty list to store game data
k = 0

# Initialize df_2025 as an empty DataFrame with appropriate columns
df_2024 = pd.DataFrame(columns=['HREF', 'team (W or L references this team)', 'date', 'opponent', 'outcome', 'score'])

# Loop until all team names are in df_2025
while len(df_2024['team (W or L references this team)'].unique()) < len(df['Team_name'].unique()):
    # Find teams that are not yet in df_2025
    remaining_teams = df[~df['Team_name'].isin(df_2024['team (W or L references this team)'].unique())]
    
    # Ensure the loop only proceeds if there are remaining teams to process
    #if remaining_teams.empty:
     #   break
    
    for i, row in remaining_teams.iterrows():
        try:
            team_schedule_link = row['Schedule_link']  # Get the Schedule link directly from the row
        
            driver.get(team_schedule_link)

            # Allow the page to load (Using WebDriverWait would be more efficient here)
            time.sleep(1)

            # Find the table containing the schedule
            table = driver.find_elements(By.CLASS_NAME, "Table__TBODY")

            if not table:
                game_data['HREF'] = team_schedule_link
                game_data['team (W or L references this team)'] = row['Team_name']
                game_data['date'] = 'No Data Available'
                game_data['opponent'] = np.null
                game_data['outcome'] = np.null
                game_data['score'] = np.null

                data.append(game_data)
                continue  # Skip if the table is not found

            lines = table[0].text.partition('\n')[2].split('\n')

            # Iterate over the lines and extract information
            for j in range(1, len(lines), 4):  # Process every 4th line since the structure repeats every 4 lines
                game_data = {}

                try:
                    date = lines[j].strip()
                    opponent = lines[j+2].strip()
                    outcome = lines[j+3].partition(' ')[0][0]  # 'W' or 'L'
                    score = lines[j+3].partition(' ')[0][1:]

                    if date == 'DATE OPPONENT TIME TV TICKETS':
                        break  # Break if we reach the end of the data section

                    # Collect the game data
                    game_data['HREF'] = team_schedule_link
                    game_data['team (W or L references this team)'] = row['Team_name']
                    game_data['date'] = date
                    game_data['opponent'] = opponent
                    game_data['outcome'] = outcome
                    game_data['score'] = score

                    data.append(game_data)
                except IndexError:
                    pass  # Skip in case of an index error (to avoid crashing)

        except Exception as e:
            print(f"Error processing {row['Team_name']}: {e}")

    # Once all teams in the loop are processed, concatenate the collected data into df_2025
    df_2024 = pd.concat([df_2024, pd.DataFrame(data)], ignore_index=True)
    if k == 0:
        # df_2025.to_csv(path_or_buf='2025_NBA_full_schedule.csv')
        print('Hey, listen') # Print statement confirms the script is working
        k += 1

# Print the resulting DataFrame
print(df_2024)

# Close the browser
driver.quit()


Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)
Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)
Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)


Error processing Mercyhurst Lakers: name 'np' is not defined
Error processing West Georgia Wolves: name 'np' is not defined
Hey, listen
                                                                                                               HREF  \
0       https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/season/2024/abilene-christian-wildcats   
1       https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/season/2024/abilene-christian-wildcats   
2       https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/season/2024/abilene-christian-wildcats   
3       https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/season/2024/abilene-christian-wildcats   
4       https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/season/2024/abilene-christian-wildcats   
...                                                                                                             ...   
631622                https://w

In [39]:
df_2024 = df_2024.drop_duplicates()

In [41]:
df_2024['outcome'] = df_2024['outcome'].replace({'W': 1, 'L': 0})
df_2024.info()


<class 'pandas.core.frame.DataFrame'>
Index: 12046 entries, 0 to 631626
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   HREF                                12046 non-null  object
 1   team (W or L references this team)  12046 non-null  object
 2   date                                12046 non-null  object
 3   opponent                            12046 non-null  object
 4   outcome                             12046 non-null  object
 5   score                               12046 non-null  object
dtypes: object(6)
memory usage: 658.8+ KB


In [43]:
df_2024 = df_2024.reset_index(drop=True)

In [45]:
df_2024.to_csv(path_or_buf='2024_NBA_full_schedule.csv')

Dataframe is saved as a csv in a local file.